In [4]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, DoubleType,MapType, StringType,ArrayType, FloatType, TimestampType, IntegerType
from cerebralcortex.core.datatypes import DataStream
from cerebralcortex.core.metadata_manager.stream.metadata import Metadata, DataDescriptor, \
ModuleMetadata
from scipy import signal
from scipy.interpolate import interp1d
from cerebralcortex import Kernel
from scipy.stats import skew,kurtosis,mode
from collections import Counter
import pandas as pd
CC = Kernel("/home/jupyter/cc3_conf/", study_name='mperf')

/cerebralcortex/kessel_jupyter_virtualenv/cc33/lib64/python3.6/site-packages/cerebralcortex/core/data_manager/raw/data.py:67: DeprecationWarning: pyarrow.hdfs.connect is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  self.fs = pa.hdfs.connect(self.hdfs_ip, self.hdfs_port)


In [5]:
def get_data_for_saving(data,
                        data_acl,
                        stream_name = 'org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity',
                        acl_stream_name = 'org.md2k.feature.motionsensehrv.decoded.rightwrist.all',
                        activities = ['Walking'],
                        window_size = 10,
                        base_window_size  = 10,
                        prediction_name = 'prediction',
                        minutes = 1500):
    data = data._data
    data = data.select('localtime','timestamp','day',prediction_name,'user','version','start', 'end','std')
    
#     groupbycols = ['user','version','day',F.window('timestamp',windowDuration=str(window_size)+' seconds', startTime='0 seconds')]
#     data_windowed = data.groupBy(groupbycols).agg(F.collect_list('prediction')).withColumnRenamed('collect_list(prediction)','prediction')
#     data_windowed = data_windowed.filter(F.size(F.col('prediction'))==window_size//base_window_size)
#     def get_most_frequent(a):
#         return Counter(a).most_common()[0][0]
#     qfunction = F.udf(get_most_frequent,StringType())
#     data_windowed = data_windowed.withColumn('prediction',qfunction(data_windowed['prediction']))
#     data_windowed = data.filter(F.col(prediction_name).isin(activities))
    
    if activities[0] not in ['std']:
        data_windowed  = data.filter(F.col(prediction_name).isin(activities))
        data_windowed = data_windowed.withColumn('time',F.col('timestamp').cast('double'))
    else:
        data_windowed  = data.filter(F.col('std')>=.21)
        data_windowed = data_windowed.withColumn('time',F.col('timestamp').cast('double'))
    
    n = int(minutes*60/base_window_size)
    schema = data_windowed.schema
    @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
    def get_user_data(df):
        df = df.sort_values('time').reset_index(drop=True)
        df = df[:n]
        return df
    
#     data_windowed = data_windowed.groupBy('user').apply(get_user_data).drop('time')
    data_windowed = data_windowed.select('localtime','timestamp','day',prediction_name,'user','version',F.struct('start', 'end').alias('window'))
    
    
    if data_windowed.count()<30*60/base_window_size:
        return pd.DataFrame([],columns=list('abcdefgh'))
#     data_windowed.printSchema()
#     print(data_windowed.count())
    
    data_acl = data_acl.select('localtime','timestamp','aclx','acly','aclz','user','version')
    data_acl = data_acl.withColumn('time',F.col('timestamp').cast('double'))
    data_acl = data_acl.withColumn('data',F.array('time','aclx','acly','aclz')).drop('time','aclx','acly','aclz')
    groupbycols = ['user','version',F.window('timestamp',windowDuration=str(window_size)+' seconds', startTime='0 seconds')]
    data_acl_windowed = data_acl.groupBy(groupbycols).agg(F.collect_list('data')).withColumnRenamed('collect_list(data)','data')
#     data_acl_windowed.printSchema()
#     data_windowed.printSchema()
    data_joined = data_windowed.join(data_acl_windowed.drop('version'),on=['user','window'],how='inner')
#     data_joined.printSchema()
    
    def interpolate_acl(a,window_size=20,fs_now=25,fs_new=25):
        x_now = np.linspace(0,window_size,a.shape[0])
        f = interp1d(x_now,a,axis=0,fill_value='extrapolate')
        x_new = np.linspace(0,window_size,window_size*fs_new)
        return f(x_new)
    
    schema = StructType([
        StructField("version", IntegerType()),
        StructField("user", StringType()),
        StructField("localtime", TimestampType()),
        StructField("timestamp", TimestampType()),
        StructField("start", TimestampType()),
        StructField("end", TimestampType()),
        StructField("data", ArrayType(DoubleType())),
        StructField("day", StringType()),
        StructField("prediction", StringType())
    ])

    @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
    def interpolate_data(df):
        df['data'] = df['data'].apply(lambda b:np.array([np.array(a) for a in b]).reshape(len(b),4))
        df['data'] = df['data'].apply(lambda a:a[a[:,0].argsort()])
        df['data'] = df['data'].apply(lambda a:a[:,1:].reshape(a.shape[0],3))
        df['data'] = df['data'].apply(lambda a:interpolate_acl(a).reshape(-1))
        return df
    data_joined = data_joined.withColumn('start',F.col('window').start)
    data_joined = data_joined.withColumn('end',F.col('window').end).drop('window')
    data_joined = data_joined.groupBy(['user','version','day']).apply(interpolate_data)
#     return data_joined.toPandas()
    schema = data_joined.schema
    stream_metadata = Metadata()
    stream_name  = stream_name+'.'+str(window_size)+'.secs.'+str(activities[0]).lower()
    print(stream_name)
    stream_metadata.set_name(stream_name).set_description("ACL data saving for REID model, window size = "+str(window_size)+' secs')
    for field in schema.fields:
        stream_metadata.add_dataDescriptor(
            DataDescriptor().set_name(str(field.name)).set_type(str(field.dataType))
        )
    stream_metadata.add_module(
        ModuleMetadata().set_name("ACL data saving for REID model, window size = "+str(window_size)+' secs'+str(activities[0]).lower()) \
        .set_attribute("url", "https://md2k.org").set_author(
            "Md Azim Ullah", "mullah@memphis.edu"))
    ds = DataStream(data=data_joined,metadata=stream_metadata)
    return ds

In [ ]:
base_window_size = 20
import pickle
import os
stream_name = 'org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity.all.three'
# activities = ['Driving','Sitting','Stairs','Walking']
activities =  ['Stationery','Stairs','Exercise','Walking','Sports']
activities = ['std']
window_size = 20
acl_stream_name = 'org.md2k.feature.motionsensehrv.decoded.rightwrist.all'
# user_id1 = '3ca3dbf5-2390-409e-bd2c-c9f23a255e75'
# users = pickle.load(open('./data/users.p','rb'))

directory = './data/'



for activity in activities[::-1]:
    activity1 = activity
#     if not os.path.isdir(directory+str(base_window_size)+'/'+activity1):
#         os.makedirs(directory+str(base_window_size)+'/'+activity1)
#     for user_id in users[:1]:
#     user_id = users[0]
    data = CC.get_stream(stream_name)
    data_acl = CC.get_stream(acl_stream_name)
    df = get_data_for_saving(data,
                            data_acl,
                            stream_name = stream_name,
                            acl_stream_name = acl_stream_name,
                            activities = [activity],
                            window_size = window_size,
                            base_window_size  = base_window_size)
    CC.save_stream(df,overwrite=True)
#     print(df.shape)
        
#         print(user_id)
#         if df.shape[0]/6 < 30:
#             continue
#         pickle.dump(df,open(directory+str(base_window_size)+'/'+activity1+'/'+user_id+'.p','wb'))
#         print(user_id,df.shape,activity)

/cerebralcortex/kessel_jupyter_virtualenv/cc33/lib64/python3.6/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity.all.three.20.secs.std


In [ ]:
shutil.rmtree('./data/10/Sitting_test/') 

In [ ]:
len(df['data'])/6
org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity.10.secs.driving
org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity.10.secs.driving

In [ ]:
data = CC.get_stream(stream_name+'.'+str(window_size)+'.secs')
df = data._data.toPandas()

In [ ]:
import pickle
pickle.dump(df,open('./data/right_wrist/'+user_id+'.p','wb'))

In [ ]:
data_final.drop('data').show(1,False)

In [ ]:
data = CC.get_stream('org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity.10.secs')

In [ ]:
data.count()

In [ ]:
import pickle
filepath = './data/walking_10/right_wrist/'
stream_name = 'org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity.10.secs'
users = pickle.load(open('./data/users.p','rb'))
for i,user in enumerate(users):
    df = CC.get_stream(stream_name,user_id=user)
    data  = df.toPandas()
    pickle.dump(data,open(filepath+user,'wb'))
    print(i,end=',')

In [ ]:
import pandas as pd




In [ ]:
import shutil

In [ ]:
maxmean_score = pd.read_json('maxmean_10.json')
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':15})
plt.figure(figsize=(20,10))
sns.heatmap(maxmean_score,annot=True,fmt='.2f',cmap='Reds',linewidth=1,linecolor='black')
plt.title('Max Mean Boosting')
plt.xlabel('Train data per user, minutes')
plt.ylabel('Test data length, minutes')
plt.tight_layout()
plt.savefig('./images/maxmean_boosting.png',dps=1e6)
plt.show()

In [ ]:
maxmean_score = pd.read_json('majority_10.json')
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':15})
plt.figure(figsize=(20,10))
sns.heatmap(maxmean_score,annot=True,fmt='.2f',cmap='Reds',linewidth=1,linecolor='black')
plt.title('Majority Boosting')
plt.xlabel('Train data per user, minutes')
plt.ylabel('Test data length, minutes')
plt.tight_layout()
plt.savefig('./images/majority_boosting.png',dps=1e6)
plt.show()

In [ ]:
df.loc[0]['data']
users
import shutil
import pickle
import os
hours = []
for f in os.listdir('./data/right_wrist/'):
    data = pickle.load(open('./data/right_wrist/'+f,'rb'))
    hours.append(data.shape[0]*20/3600)
data.shape[0]*20/3600
import matplotlib.pyplot as plt

from collections import Counter

import numpy as np
Counter(np.floor(hours))